# Calibration
## Goal Map pixels to inches / feet
We need to transfer observations from x-y pixel space to physical units

## imports


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

## functions


### frame space

In [ ]:
def get_frame(cap, n):
    cap.set(cv2.CAP_PROP_POS_FRAMES, n)
    return cap.read()

def get_video_parameters(cap):
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    return fps, width, height, frame_count

def find_circles_hough(frame: np.ndarray, 
    param2: int=7,
    min_radius: int=5, max_radius: int=8,
) -> np.ndarray:
    """ Finds circles in a cv2 frame."""
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)        # create grayscale copy
    return cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, dp=1, minDist=20,
                        param1=50, param2=param2, 
                        minRadius=min_radius, maxRadius=max_radius)

In [ ]:
# ballphsics/vision/utils.py
def extract_vertical_slice(frame: np.ndarray, x_start: int, x_end: int) -> np.ndarray:
    """Extract a vertical slice from frame between x_start and x_end."""
    return frame[:, x_start:x_end]

def extract_horizontal_slice(frame: np.ndarray, y_start: int, y_end: int) -> np.ndarray:
    """Extract a horizontal slice from frame between y_start and y_end."""
    return frame[y_start:y_end, :]

def extract_region(frame: np.ndarray, x_start: int, x_end: int, y_start: int, y_end: int) -> np.ndarray:
    """Extract a rectangular region from frame."""
    return frame[y_start:y_end, x_start:x_end]


In [ ]:
def rotate_points(points, angle_degrees):
    angle_rad = np.radians(angle_degrees)
    cos_a, sin_a = np.cos(angle_rad), np.sin(angle_rad)
    rotation_matrix = np.array([[cos_a, -sin_a], [sin_a, cos_a]])
    return points @ rotation_matrix.T

### visualization

In [ ]:
# ballphysics/visualization/plotting.py
def view_frame(frame, h=10, w=12):
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(h, w))
    plt.imshow(frame_rgb)
    plt.gca().xaxis.set_ticks_position('both')
    plt.tick_params(top=True, labeltop=True)
    plt.show()


def show_frames_side_by_side(frame1, frame2, title1='Frame 1', title2='Frame 2'):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 10))
    
    frame1_rgb = cv2.cvtColor(frame1, cv2.COLOR_BGR2RGB)
    frame2_rgb = cv2.cvtColor(frame2, cv2.COLOR_BGR2RGB)
    
    ax1.imshow(frame1_rgb)
    ax1.set_title(title1)
    ax1.axis('on')
    ax1.xaxis.set_ticks_position('both')
    ax1.xaxis.set_ticks_position('both')
    ax1.tick_params(top=True, labeltop=True)

    ax2.imshow(frame2_rgb)
    ax2.set_title(title2)
    ax2.axis('on')
    ax2.xaxis.set_ticks_position('both')
    ax2.tick_params(top=True, labeltop=True)
    
    plt.tight_layout()
    plt.show()

### analysis


In [ ]:
def rotate_points(points, angle_degrees):
    angle_rad = np.radians(angle_degrees)
    cos_a, sin_a = np.cos(angle_rad), np.sin(angle_rad)
    rotation_matrix = np.array([[cos_a, -sin_a], [sin_a, cos_a]])
    return points @ rotation_matrix.T

## video path


In [ ]:
freefall_path = "../data/freefall.mp4"

## video characteristics

In [ ]:
cap = cv2.VideoCapture(freefall_path)

In [ ]:
fps, width, height, frame_count = get_video_parameters(cap)
"width and height", width, height, "frame count and fps", frame_count, fps

## is pegboard backdrop vertical
- Use the pegboard to allow a transform from pixel space to physical space.
- Ideally it should be vertical and the camera should be horizontal
- If possible confirm vertical pegboard with level or plumb
- Confirm camera plumb from photo before taking videos.

### Not in this video
- From measurements of the ball trajectory - camera was 2.14° out of level


### camera was 2.14° out of level


In [ ]:
ret, frame = get_frame(cap, 0)


In [ ]:
#| export
view_frame(frame)

#### Characteristics of the board
- Attached to metal studs on each side with screws
- Wood supports to keep the metal studs square
- Outside peg holes show the metal
- Where the wood supports are the holes are different
- Other holes have mixed lighting levels behind the backdrop

In [ ]:
f_top = extract_horizontal_slice(frame, 0, 300)
f_bot = extract_horizontal_slice(frame, height - 300, height)


In [ ]:
show_frames_side_by_side(f_top, f_bot, "Top", "Bottom")


#### further crop vertically to examine the left edge

In [ ]:
left = 550
right = 750
edge_top = f_top[:, left:right]
edge_bot = f_bot[:, left:right]
show_frames_side_by_side(edge_top, edge_bot, "edge top", "edge bottom")

## Crop the full frame vertically to locate the peg holes

In [ ]:
left = 600
right = 1200

In [ ]:
vertical_slice = frame[:, left:right]
view_frame(vertical_slice, h=4, w=800)

### Refine Hough parameters to identify circles

#### Eyeballing the slice above - Try 8 as a maximum radius

In [ ]:
min_radius = 5
max_radius = 8

print(f"Radius  min {min_radius} max {max_radius}")
for par2 in range(1,9):
    circles = find_circles_hough(vertical_slice, param2=par2, min_radius=min_radius, max_radius=max_radius)
    circle_count = 0
    if circles is not None:
        circle_count = len(circles[0])
    print(f"param2 value {par2:2d}  - No of circles found {circle_count:4d}")

#### Experiment with par2
Maximize positives, minimize false positives

In [ ]:
par2 = 7
min_radius = 5
max_radius = 8
# result = input("param2")
# par2 = int(result)
# max_radius = input("max_radius")
# max_radius = int(max_radius)

#### Find circles in a vertical slice

In [ ]:
circles = find_circles_hough(vertical_slice, param2=7, min_radius=5, max_radius=8)
circle_count = len(circles)

In [ ]:
def show_circles_found(frame, circles):
    green = (0, 255, 0)
    red = (0, 0, 255)
    result = frame.copy()
    if circles is not None:
        circles = np.uint16(np.around(circles))
        for i in circles[0, :]:
            cv2.circle(result, (i[0], i[1]), i[2], green, 2)
            cv2.circle(result, (i[0], i[1]), 2, red, 3)
    view_frame(result, h=4, w=800)

#### Distribution of circle radii

In [ ]:
if circles is not None:
    radii = circles[0, :, 2]
    plt.hist(radii, bins=20)
    plt.xlabel('Radius')
    plt.ylabel('Count')
    plt.show()

#### Use k-means clustering to select clusters of vertical holes

In [ ]:
from sklearn.cluster import KMeans

# Extract circle centers
centers = circles[0, :, :2]  # x, y coordinates

# Cluster by x-coordinate (adjust n_clusters based on how many columns you expect)
nclusters = 16
kmeans = KMeans(n_clusters=nclusters, random_state=42)
labels = kmeans.fit_predict(centers[:, 0].reshape(-1, 1))

# Fit a line to each cluster
result_lines = vertical_slice.copy()
for cluster_id in range(nclusters):
    cluster_points = centers[labels == cluster_id]
    if len(cluster_points) > 1:
        # Fit line: y = mx + b
        coeffs = np.polyfit(cluster_points[:, 1], cluster_points[:, 0], 1)
        
        # Draw the line
        y_min, y_max = int(cluster_points[:, 1].min()), int(cluster_points[:, 1].max())
        x_min = int(coeffs[0] * y_min + coeffs[1])
        x_max = int(coeffs[0] * y_max + coeffs[1])
        cv2.line(result_lines, (x_min, y_min), (x_max, y_max), (255, 0, 0), 2)

view_frame(result_lines, h=4, w=800)

In [ ]:
print(np.bincount(labels))


#### Show results - angle from horizontal and hole spacing in pixels

In [ ]:
slopes = []
for cluster_id in range(nclusters):
    cluster_points = centers[labels == cluster_id]
    if len(cluster_points) > 1:
        coeffs = np.polyfit(cluster_points[:, 1], cluster_points[:, 0], 1)
        slopes.append(coeffs[0])

angles = np.degrees(np.arctan(slopes))
mean_angle = np.mean(angles)
print(f"Angles from vertical: {angles}")
print(f"Mean angle: {mean_angle:.3f} degrees")
print(f"Std dev: {np.std(angles):.3f} degrees")
all_gaps = []
for cluster_id in range(nclusters):
    cluster_points = centers[labels == cluster_id]
    if len(cluster_points) > 1:
        # Sort by y-coordinate
        cluster_sorted = cluster_points[cluster_points[:, 1].argsort()]
        # Rotate points
        rotated = rotate_points(cluster_sorted, mean_angle)
        # Calculate gaps
        gaps = np.fabs(np.diff(rotated[:, 1]))
        # Keep only gaps < 50 (1-inch spacings)
        small_gaps = gaps[gaps < 50]
        all_gaps.extend(small_gaps)

x_pixels_per_inch = np.mean(all_gaps)
print(f"Pixels per inch - x direction: {x_pixels_per_inch:.2f}")
print(f"Number of gaps used: {len(all_gaps)}")

## Crop the full frame horizontally to locate the peg holes

In [ ]:
left   =  600
right  = 1200
top    =  100
bottom = 3500
horizontal_slice1 = frame[top:bottom, left:right]

In [ ]:
view_frame(horizontal_slice1, h=4, w=800)

In [ ]:
# left   =  600
# right  = 1200
# top    = 2800
# bottom = 3500
# horizontal_slice2 = frame[top:bottom, left:right]

In [ ]:
view_frame(horizontal_slice1, h=4, w=800)

In [ ]:
min_radius = 5
max_radius = 8

print(f"Radius  min {min_radius} max {max_radius}")
for par2 in range(6,7):
    circles = find_circles_hough(horizontal_slice1, param2=par2, min_radius=min_radius, max_radius=max_radius)
    circle_count = 0
    if circles is not None:
        circle_count = len(circles[0])
    print(f"param2 value {par2:2d}  - No of circles found {circle_count:4d}")
# circles = find_circles_hough(vertical_slice, param2=7, min_radius=5, max_radius=8)
# circle_count = len(circles)
show_circles_found(horizontal_slice1, circles)

#### Distribution of circle radii

In [ ]:
if circles is not None:
    radii = circles[0, :, 2]
    plt.hist(radii, bins=20)
    plt.xlabel('Radius')
    plt.ylabel('Count')
    plt.show()

#### Use k-means clustering to select clusters of horizontal holes

In [ ]:
nclusters = 60
# Extract circle centers
centers = circles[0, :, :2]  

# x, y coordinates# Cluster by y-coordinate (for horizontal rows)
kmeans = KMeans(n_clusters=nclusters, random_state=42)
labels = kmeans.fit_predict(centers[:, 1].reshape(-1, 1))

good_clusters = []
# Fit a line to each cluster
result_lines = horizontal_slice1.copy()
for cluster_id in range(nclusters):
    cluster_points = centers[labels == cluster_id]
    if len(cluster_points) > 6:
        # Fit line: x = my + b
        coeffs = np.polyfit(cluster_points[:, 0], cluster_points[:, 1], 1)
        
        x_min, x_max = int(cluster_points[:, 0].min()), int(cluster_points[:, 0].max())
        y_min = int(coeffs[0] * x_min + coeffs[1])
        y_max = int(coeffs[0] * x_max + coeffs[1])
        cv2.line(result_lines, (x_min, y_min), (x_max, y_max), (255, 0, 0), 2)
        
        y_variance = np.fabs(y_max - y_min)
        # print(f"Cluster {cluster_id}: {coeffs} {len(cluster_points)} points, y_variance: {y_variance:.2f}")

        if y_variance < 10 and coeffs[0] > 0:
        # Draw the line
            x_min, x_max = int(cluster_points[:, 0].min()), int(cluster_points[:, 0].max())
            y_min = int(coeffs[0] * x_min + coeffs[1])
            y_max = int(coeffs[0] * x_max + coeffs[1])
            cv2.line(result_lines, (x_min, y_min), (x_max, y_max), (255, 0, 0), 2)
            good_clusters.append((cluster_id, len(cluster_points)))

        # else:
        #     print(f"Excluding Cluster {cluster_id}") 

view_frame(result_lines, h=4, w=800)

In [ ]:
len(good_clusters), sum(count for _, count in good_clusters)

#### Show results - angle from vertical and hole spacing in pixels

In [ ]:
slopes = []
for good_cluster in good_clusters:
    cluster_points = centers[labels == good_cluster[0]]
    if len(cluster_points) > 1:
        coeffs = np.polyfit(cluster_points[:, 1], cluster_points[:, 0], 1)
        slopes.append(coeffs[0])

angles = np.degrees(np.arctan(slopes))
mean_angle = np.mean(angles)
print(f"Angles from vertical: {angles}")
print(f"Mean angle: {mean_angle:.3f} degrees")
print(f"Std dev: {np.std(angles):.3f} degrees")


all_gaps = []
for good_cluster in good_clusters:
    cluster_points = centers[labels == good_cluster[0]]

    if len(cluster_points) > 1:
        # Sort by y-coordinate
        cluster_sorted = cluster_points[cluster_points[:, 1].argsort()]
        # Rotate points
        rotated = rotate_points(cluster_sorted, mean_angle)
        # Calculate gaps
        gaps = np.fabs(np.diff(rotated[:, 1]))
        # Keep only gaps < 50 (1-inch spacings)
        small_gaps = gaps[gaps < 50]
        all_gaps.extend(small_gaps)

y_pixels_per_inch = np.mean(all_gaps)
print(f"Pixels per inch y direction: {y_pixels_per_inch:.2f}")
print(f"Number of gaps used: {len(all_gaps)}")

hmm. The one with the bad slope caused quite a nasty result in the angle.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

You're absolutely right! Those two outlier angles (around -89° and -87°) are pulling your mean way off. When most angles are around +88-89°, averaging in negative values gives you that meaningless 79.5° result.

Would you like to filter out those outlier slopes before calculating the mean angle? You could use something like removing values that are more than 2-3 standard deviations away, or just filter for angles in a reasonable range (say, between 85° and 90°).

Which approach sounds better to you?

Actually, are we measuring the angle wrong?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Good catch! Let's think about this. You're using `np.arctan(slopes)` where the slopes come from fitting x as a function of y (since you did `np.polyfit(cluster_points[:, 1], cluster_points[:, 0], 1)`).

For horizontal lines, the slope should be close to zero (x barely changes as y changes). So `arctan(slope)` should give you angles close to 0°, not 90°.

But you're getting angles around 88-89°, which suggests the lines are nearly vertical - which doesn't make sense for horizontal rows of holes!

What do you think is happening here? Should you be calculating the angle differently, or fitting the line differently?

The reason for the slope is to adjust the pixels per inch. So we really want the deviation from 90, right?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Exactly! For horizontal lines, you want to measure how far they deviate from being perfectly horizontal (0° slope), not from vertical.

So instead of just using `mean_angle`, you probably want something like `90 - mean_angle` to get the actual tilt from horizontal. Or even simpler - just use the slopes directly when rotating, since a small slope already tells you the deviation.

Does that make sense for what you're trying to do with the rotation correction?

I copied this from the vertical version
```
slopes = []
for cluster_id in range(nclusters):
    cluster_points = centers[labels == cluster_id]
    if len(cluster_points) > 1:
        coeffs = np.polyfit(cluster_points[:, 1], cluster_points[:, 0], 1)
        slopes.append(coeffs[0])

angles = np.degrees(np.arctan(slopes))
mean_angle = np.mean(angles)
print(f"Angles from vertical: {angles}")
print(f"Mean angle: {mean_angle:.3f} degrees")
print(f"Std dev: {np.std(angles):.3f} degrees")
all_gaps = []
for cluster_id in range(nclusters):
    cluster_points = centers[labels == cluster_id]
    if len(cluster_points) > 1:
        # Sort by y-coordinate
        cluster_sorted = cluster_points[cluster_points[:, 1].argsort()]
        # Rotate points
        rotated = rotate_points(cluster_sorted, mean_angle)
        # Calculate gaps
        gaps = np.diff(rotated[:, 1])
        # Keep only gaps < 50 (1-inch spacings)
        small_gaps = gaps[gaps < 50]
        all_gaps.extend(small_gaps)

pixels_per_inch = np.mean(all_gaps)
print(f"Pixels per inch: {pixels_per_inch:.2f}")
print(f"Number of gaps used: {len(all_gaps)}")
```
I think somewhere I did not correct it well. (Also it was filtered differently)

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Right! So for the vertical version, you were measuring deviation from vertical (which makes sense - vertical lines should have infinite slope, so you measure the angle from vertical).

For horizontal lines, you need to think about it differently. When you fit `x = m*y + b` for horizontal lines:
- A perfectly horizontal line has slope m ≈ 0
- The angle from horizontal is `arctan(m)`, which should be close to 0°

So you probably want to either:
1. Change the print statement to say "Angles from horizontal" and expect values near 0°, OR
2. Calculate `90 - angle` if you want to express it as deviation from vertical

Also, when rotating, you'd rotate by the angle from horizontal (the small angle), not 90-something degrees.

Which interpretation makes more sense for your calibration?

In [ ]:
print(f"Pixels per inch x direction: {x_pixels_per_inch:.2f}")
print(f"Pixels per inch y direction: {y_pixels_per_inch:.2f}")
